In [4]:
# Import modules and subroutines:
using ConfParser
using LaTeXStrings
include("../subroutines/Subroutines.jl");

3.10.9


In [2]:
# Load data and config parameters:
fid = h5open("../datasets/tnqe_data/h2o_test_2023-05-22.h5", "r")

conf_path = read(fid["conf_path"])

conf, op_list, gp_list, swp_list = FetchConfig("../$(conf_path)")

# Load chemical data:
cdata_list = []

for m=1:conf.nmol

    cdata = ReadIn("../"*conf.pyscf_paths[m])[1];
    push!(cdata_list, cdata)
    PrintChemData(cdata)
    
end

Molecule name: h2o_1p8A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h2o_double_dissoc/1p8.xyz
RHF energy: -74.51129616870286
FCI energy: -74.79063381243297


In [3]:
# Collect the potential energy curves:

e_rhf_vec = [cdata_list[m].e_rhf for m=1:length(cdata_list)]
e_fci_vec = [cdata_list[m].e_fci for m=1:length(cdata_list)]

atz_labels = conf.atz_name

e_atz_vecs = []
for a=1:conf.n_atz
    e_atz_vec = [read(fid["$(conf.mol_names[m])/$(conf.atz_name[a])/E"])[1] + cdata_list[m].e_nuc for m=1:conf.nmol]
    push!(e_atz_vecs, e_atz_vec)
    display(e_atz_vec)
end

1-element Vector{Float64}:
 -74.79038525890245

1-element Vector{Float64}:
 -74.59057845940445

1-element Vector{Float64}:
 -74.79024228461056

1-element Vector{Float64}:
 -74.789002155791

In [4]:
display(atz_labels)

4-element Vector{String}:
 "dmrg8"
 "dmrg3"
 "nomps"
 "tnqe"

In [7]:
# Plot the potential energy curves:

# x-axis information:
x_vals = [1.8,2.0,2.2,2.4,2.6,2.8,3.0]
x_label = L"r\quad(\AA)"

# Plot title:
title = "Double-dissociation of H2O in STO-3G basis"

bond_lengths = [1.8,2.0,2.2,2.4,2.6,2.8,3.0]

using LaTeXStrings

plot(
    x_vals, 
    e_rhf_vec, 
    color=RGB(0.7,0.7,0.7), 
    linewidth=2.0,
    label="RHF",
    title=title,
    titlefontsize=11.0,
    titlefont=font(14,"Computer Modern"),
    xguide=x_label,
    yguide=L"E\quad(\textrm{Ha.})",
    size=(700,500)
)

plot!(
    x_vals, 
    e_fci_vec, 
    color=RGB(0.3,0.3,0.3), 
    linewidth=2.0,
    label="FCI"
)

for a=1:conf.n_atz-2
    
    plot!(
        x_vals, 
        e_atz_vecs[a], 
        linewidth=2.0,
        markersize=4,
        markershape=:circle,
        label=atz_labels[a]
    )
    
end


savefig("../plots/test.pdf")

LoadError: BoundsError: attempt to access 1-element Vector{Float64} at index [1:7]

In [ ]:
plot!(
    bond_lengths, 
    e_dmrg8_vec, 
    color=RGB(0.4,0.4,1.0), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=8"
)

plot!(
    bond_lengths, 
    e_dmrg3_vec, 
    color=RGB(0.3,0.8,0.5), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=3"
)

plot!(
    bond_lengths, 
    e_nomps_vec, 
    color=color=RGB(1.0,0.8,0.3), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="Cl. NOMPS; M=5, m=3"
)

plot!(
    bond_lengths, 
    e_tnqe_vec, 
    color=color=RGB(1.0,0.4,0.4), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="TNQE (NOMPS); M=5, m=3"


In [9]:
parse(Float64, "1e-1")

0.1